In [516]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [517]:
def format(df):
    df["Date"] = pd.to_datetime(df["年月日"])
    df.set_index('Date', inplace = True)
    df.drop(columns = ['代號', '名稱', '年月日'], inplace = True)
    df.columns = [['Open', 'Close', 'Volume', 'Capitalization']]
    df['Cost'] = 0.0
    df['Share_hold'] = 0.0
    df['Value'] = 0.0
    df['Log_return'] = 0.0

    trade_days = [0]
    investment_days = (
        df.groupby(df.index.to_period("M"))
        .apply(lambda x: x.iloc[trade_days] if len(x) > max(trade_days) else x.iloc[:len(x)])
    )
    investment_days['Date'] = investment_days.index.get_level_values(1)
    trade_dates = set(pd.to_datetime(investment_days['Date'].squeeze().tolist()))

    return df, trade_dates

In [518]:
def iterate(df, trade_dates, investment_amount = 3000):
    for i in range(len(df)):
        if df.index[i] in trade_dates:
            shares_bought = investment_amount // df['Open'].iloc[i].values
            df.loc[df.index[i], 'Cost'] = shares_bought * df['Open'].iloc[i].values
            df.loc[df.index[i], 'Share_hold'] = df['Share_hold'].iloc[i - 1].values + shares_bought
            
        else:
            df.loc[df.index[i], 'Share_hold'] = df['Share_hold'].iloc[i - 1].values

        if i == 0:
            continue

        df.loc[df.index[i], 'Value'] = df['Share_hold'].iloc[i].values * df['Open'].iloc[i].values
        df.loc[df.index[i], 'Log_return'] = np.log(df['Open'].iloc[i].values / df['Open'].iloc[i - 1].values)
        df.loc[df.index[i], 'Return'] = np.exp(df['Log_return'].iloc[i].values) - 1

    return df

In [ ]:
folder_path = '/Users/xinc./Documents/GitHub/Quant/data/heather'
for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)

    if os.path.isdir(subfolder_path):
        for filename in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, filename)
            
            if filename.endswith('.xlsx'):
                df = pd.read_excel(file_path)
                df, trade_dates = format(df)
                df = iterate(df, trade_dates)
                df.to_excel(file_path)

/var/folders/np/82kc955j7g9_6x0dw88zjzw40000gn/T/ipykernel_62130/1397948487.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.003056548482548874' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df.index[i], 'Log_return'] = np.log(df['Open'].iloc[i].values / df['Open'].iloc[i - 1].values)
/var/folders/np/82kc955j7g9_6x0dw88zjzw40000gn/T/ipykernel_62130/1397948487.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5493.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df.index[i], 'Value'] = df['Share_hold'].iloc[i].values * df['Open'].iloc[i].values
/var/folders/np/82kc955j7g9_6x0dw88zjzw40000gn/T/ipykernel_62130/1397948487.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in